In [ ]:
from copy import deepcopy
from functools import partial, reduce
import os
from operator import or_, and_, contains

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, MATCH, ALL
from dash.exceptions import PreventUpdate
from dateutil.parser import parse
import django
from django.db.models import Q
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from toolz import keyfilter, valfilter, merge, isiterable, get_in

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway and is not presently being used

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
from plotter.graph import *
# from plotter.views import *
# from plotter.forms import *
from utils import (rows, columns, qlist, keygrab, in_me,
                   particular_fields_search, interval_search, pickitems,
                   pickctx)

In [ ]:
app = dash.Dash()
CACHE_CONFIG = {'CACHE_TYPE': 'filesystem', 'CACHE_DIR': './.cache'}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

# we're using this to create thread-safe global values.
# also caching relatively expensive things like database lookups to the filesystem.
# is this actually slower? maybe. if so we will supplement with storing
# data in divs.
# we could instead just memoize to memory using lru_cache or similar


cset = cache_set(cache)
cget = cache_get(cache)

In [ ]:
fig = main_graph()
graph_function = scatter
spec_model = MSpec
obs_model = MObs

# active queryset is explicitly stored in global cache
cset('queryset', spec_model.objects.all().prefetch_related("observation"))

# these lists of input objects are defined here primarily to avoid excessive
# repetition in app structure definition and function calls.
x_inputs = [
    Input('filter-1-x', 'value'),
    Input('filter-2-x', 'value'),
    Input('filter-3-x', 'value'),
    Input('axis-option-x', 'value'),
]
y_inputs = [
    Input('filter-1-y', 'value'),
    Input('filter-2-y', 'value'),
    Input('filter-3-y', 'value'),
    Input('axis-option-y', 'value')
]

In [ ]:
# insert 'global' variables for this app into functions.
# note that insertion of spec_model into functions may end up being
# a way to generate separate function 'namespaces'
# in the possible case of wanting to mix mastcam / z data 
# within a single app instance

recalculate_graph = partial(
    recalculate_graph, 
    x_inputs=x_inputs,
    y_inputs=y_inputs,
    cget=cget,
    cset=cset,
    graph_function = graph_function
)
update_search_options = partial(update_search_options, cget=cget)
update_queryset = partial(
    update_queryset, cget=cget, cset=cset, spec_model = spec_model
)
change_calc_input_visibility = partial(
    change_calc_input_visibility, spec_model = spec_model
)
toggle_search_input_visibility = partial(
    toggle_search_input_visibility, spec_model = spec_model
)
add_dropdown = partial(
    add_dropdown, cget=cget, spec_model=spec_model
)

In [ ]:
# memoize functions that seem to want caching
# this isn't totally working because the local version isn't mostly what's
# actually being called. doesn't deal with checks in handle_search, etc
# not a big deal right now, will optimize later

for function in [
    recalculate_graph,
    handle_graph_search,
    qlist,
    make_axis,
    update_search_options,
    update_queryset
    ]:
    function = cache.memoize()(function)

In [ ]:
app.layout = html.Div(children = [
    html.Div(children = [
        axis_value_drop(spec_model, 'axis-option-x'),
        filter_drop(spec_model,'filter-1-x'),
        filter_drop(spec_model,'filter-3-x'),
        filter_drop(spec_model,'filter-2-x'),
    ]),
    html.Div(children = [
        axis_value_drop(spec_model, 'axis-option-y'),
        filter_drop(spec_model,'filter-1-y'),
        filter_drop(spec_model,'filter-3-y'),
        filter_drop(spec_model,'filter-2-y'),
    ]),
    html.Div(
        children = [dcc.Input(id='search-trigger')],
        style={"display":"none"}
    ),
    # this is initially empty and populated on page load
    html.Div(
        children = [],
        id = 'search-container'
    ),
    html.Button('add search parameter', id='add-param', n_clicks=0),
    html.Button(id='submit-search', children='Submit', n_clicks=0),
    html.Div(children = [fig], id = 'main-graph-container'),
    ])

In [ ]:
# register functions with app i/o

for axis in ['x', 'y']:
    app.callback(
        [
            Output('filter-1-'+axis, 'style'),
            Output('filter-2-'+axis, 'style'),
            Output('filter-3-'+axis, 'style'),
        ],
        [Input('axis-option-'+axis, 'value')]
    )(change_calc_input_visibility)

app.callback(
    Output('main-graph', 'figure'),
    # maybe later add an explicit recalc button?
    [*x_inputs, *y_inputs, Input('search-trigger', 'value')],
)(recalculate_graph)

app.callback(
    [
        Output({'type':'term-search', 'index':MATCH},'style'),
        Output({'type':'number-search-begin', 'index':MATCH},'style'),
        Output({'type':'number-search-end', 'index':MATCH},'style')
    ],
    [Input({'type':'field-search', 'index':MATCH},'value')],
    )(toggle_search_input_visibility)

app.callback(
    [Output({'type':'term-search', 'index':MATCH},'options'),
     Output({'type':'number-range-display','index':MATCH},'children')],
    [Input({'type':'field-search', 'index':MATCH},'value')],
    )(update_search_options)

app.callback(
        Output('search-trigger', 'value'),
        [Input('submit-search', 'n_clicks')],
        [State({'type': 'field-search', 'index': ALL}, 'value'),
        State({'type': 'term-search', 'index': ALL}, 'value'),
        State({'type': 'number-search-begin', 'index': ALL}, 'value'),
        State({'type': 'number-search-end', 'index': ALL}, 'value')
        ],
    )(update_queryset)

app.callback(
    Output('search-container', 'children'),
    [Input('add-param', 'n_clicks')],
    [State('search-container', 'children')])(add_dropdown)

In [ ]:
app.run_server()